# \*\*Supplementary Materials for: Estimating Agricultural Practices and

their Impact on Biodiversity from Agricultural Statistics: A Proof-of-Concept Study on Food Labels in France\*\*

Sarah HUET [](https://orcid.org/0000-0002-5449-6959) (CESAER UMR1041, INRAE, Institut Agro, Université Bourgogne Franche-Comté, Dijon, France)  
Clélia SIRAMI [](https://orcid.org/0000-0003-1741-3082) (Université de Toulouse, INRAE, UMR Dynafor, Castanet-Tolosan, France)  
Abdoul DIALLO (CESAER UMR1041, INRAE, Institut Agro, Université Bourgogne Franche-Comté, Dijon, France)  
Julie REGOLO (US-ODR, INRAE, Auzeville-Tolosane, France)  
Ainhoa IHASUSTA (CESBIO, Université de Toulouse, CNES/CNRS/INRAE/IRD/UPS, 18 Avenue Edouard Belin, bpi 2801, CEDEX 09, 31401 Toulouse, France)  
Ludovic ARNAUD (CESBIO, Université de Toulouse, CNES/CNRS/INRAE/IRD/UPS, 18 Avenue Edouard Belin, bpi 2801, CEDEX 09, 31401 Toulouse, France)  
Valentin BELLASSEN [](https://orcid.org/0000-0001-8581-2814) (CESAER UMR1041, INRAE, Institut Agro, Université Bourgogne Franche-Comté, Dijon, France)  
August 26, 2024

# Supplementary Methods

## Landscape variables

To take into account the landscape dimension in the BVIAS model applied to environmental labeling, we estimate three parameters: hedge area, agricultural plot size and cultural diversity. To calculate the variables necessary for estimating these parameters, we first extract from the 2020 Graphical Plot Register (RPG) all plots of farms in the 2020 FADN-AR-NAOS matched database.

To do this extraction, we use the PACAGE plot numbers as primary join key. We are faced with three situations:

-   the farm has the same PACAGE number in FADN and RA. Then the RPG plots of this PACAGE number are associated with that farm RICA-RA-INAO;

-   the PACAGE numbers differ between RICA and RA and only one of them corresponds to a RPG PACAGE number. Then the RPG plots of this PACAGE number are associated with this exploitation RICA-RA-INAO;

-   the PACAGE numbers are not entered in either the FADN or the AR, or differ between FADN and AR, and none of them corresponds to a RPG PACAGE number. Then, no parcel is associated with a FADN-AR-NAOI farm. This case is found for 443 and 318 metropolitan farms, respectively. These holdings are mainly concentrated in the OTEX viticulture, market gardening and pig and/or poultry farming.

The result of this extraction will be a subsample of the RPG that we intersect with the hedge layer of the Topo database to calculate the variables necessary for the estimation of the three parameters.

### Density of hedges

Hedge density is the ratio of the sum in linear metres of hedge to the area of the holding (UAA). For the calculation of linear lengths, we will use the same procedure as previously used in ***a similar work*** at the scale of the regions of France. For this calculation, we will distinguish four categories for each variable (hedge area and SAU): - Total - Permanent grassland (including grasslands, moors and alpine pastures) - Arboriculture (orchards and vineyards) - Other crops

### Average size of the plots

The average size of the plots is the ratio of the UAA to the number of plots. We will distinguish four categories for each variable (UAA and number of plots): - Total - Permanent Grassland - Arboriculture (orchards and vineyards) - Other crops

### Cultural diversity

We calculate two indices here, the Shannon (@eq-shannon) and the Simpson reciprocal (@eq-simpson), using the number of crops as the number of species and the surface area as the abundance. For each index (Shannon and Simpson reciprocal), we will distinguish three categories: - Total - Other crops - Other crops + permanent grassland

$$
H' = - \sum_{i=1}^{R} p_i ln p_i
$${#eq-shannon}

where $p_i$ is the ratio of the area of a crop $i \in R$ to the total area.

$$
²D = \frac{1}{ \sum^{R}_{i = 1} \bigg( \frac{n_i(n_i - 1)}{N(N-1)} \bigg)^2 }
$${#eq-simpson}

where $n$ is the area of the crop $i \in R$ and $N$ the total area.

### Average annual plant cover in number of days

From the raster data on plant cover in number of days in 2020 between 0 and 366, we calculated the average number of days in plant cover by pasture and crop category. To do this, we started by calculating the average number of days of the grazing plots using the area statistics algorithm in QGIS. Then we calculated the average number of days for each crop category by aggregating the average number of days of covered plots according to: - Total - Permanent Grassland - Arboriculture (orchards and vineyards) - Other crops

## Calibration and validation of the model by comparison with in situ biodiversity measurements

# Supplementary Tables

``` python

#| label: tbl-results_crops

library(gt)
#> Warning: le package 'gt' a été compilé avec la version R 4.4.1

tmp_table <- tmp_table0 %>%
  filter(production_type == "crop"
         & practice %in% c(tmp_practice_names_BVIAS$labels,
                           tmp_practice_names_crops$labels)) %>%
  select(!c(production_type,practice_subset)) %>%
  arrange(practice,product_name,FQS)

tmp_table %>%
  # groups
  mutate(product_name = ifelse(row_number() == 1,
                               as.character(product_name), ""),
         .by = c(practice, product_name)) %>%
  gt(groupname_col = "practice") %>%
  tab_style(
      style = list(
        cell_fill(color = "gray"),
        cell_text(weight = "bolder")
      ),
      locations = cells_row_groups(groups = everything())
    ) %>%
  # format numbers
  fmt_auto() %>%
  tab_style(
    style = list(
      cell_text(weight = "bold")
      ),
    locations = cells_body(
      columns = c(diff_sans_match,pval_tukey),
      rows = pval_tukey <= 0.05
    )
  ) %>%
  tab_style(
    style = list(
      cell_text(weight = "bold")
      ),
    locations = cells_body(
      columns = c(mean_diff_match,padjust_fdr),
      rows = padjust_fdr <= 0.05
    )
  ) %>%
  # replace NAs
  sub_missing(missing_text = "-") %>%
  # column labels
  tab_style(
    style = list(
      cell_text(weight = "bolder",size = "large")
      ),
    locations = cells_column_labels(everything())
  ) %>%
  cols_label(
    practice = "Pratiques",
    product_name = "Production",
    FQS = "Label",
    FQS_mean = "Moyenne",
    std = "Ecart-Type",
    r = "Effectif",
    diff_sans_match = "Différence avec la moyenne conventionnelle",
    pval_tukey = "Tukey HSD p-value",
    n_matched = "Effectif apparié",
    mean_diff_match = "Différence avec la moyenne contrefactuelle",
    padjust_fdr = "FDR adjusted t-test p-value"
  )
```

</div>

**Descriptive statistics and test results on estimated practices and their impact BVIAS for the production of dairy cow feed** (significant differences and corresponding p-values in bold)

**Descriptive statistics and test results on estimated practices and their impact BVIAS for dairy cow rearing** (in bold the significant differences and their p-value)

# Supplementary Figures

Figure 6: Diagramme détaillé de la modélisation des pratiques agricoles à partir des données du RICA et de leur intégration avec le modèle BVI

{#fig-diagram_detailed }